# ISSUES #
## validation set에 대해 계산한 Columnwise mean ROC AUC가 실제 테스트셋에 대해 제출했을 때 값과 차이가 많이 남

## Requirements

- pytorch
- torchtext
- pandas
- scikit-learn
- numpy
- tqdm
- gensim


In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchtext
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd 
import random
import numpy as np

## Model 설명

- embedding layer
- |
- convolutional layer (kernel = 3 x embedding dim)
- |
- leakyrelu
- |
- dropout
- |
- maxpool w.r.t time axis
- |
- fcn1 for each labels
- |
- fcn2 for each labels ( -> label output )
- | 
- sigmoid - BinaryCrossEntropyLoss

In [2]:
a = Variable(torch.randn([64,1,100,100]))
b= Variable(torch.randn([64,1, 100]))

In [3]:
a.shape, b.shape

(torch.Size([64, 1, 100, 100]), torch.Size([64, 1, 100]))

In [4]:
torch.cat([a, b], dim=3)

Variable containing:
( 0 , 0 ,.,.) = 
  8.8214e-01  8.3378e-02 -1.9106e-01  ...   1.3042e+00  7.9262e-01  2.4372e-01
  1.2094e+00 -1.2408e-01  1.5102e+00  ...   1.8979e-01 -1.1268e+00 -3.3760e-01
  8.6431e-03  1.4372e+00  7.1113e-01  ...  -9.8999e-01  2.7993e+00 -2.5664e-01
                 ...                   ⋱                   ...                
  6.4405e-01  7.2409e-01  1.4551e+00  ...   5.4108e-02  8.2792e-01  2.2840e-01
 -6.8613e-03  1.9607e+00  1.3133e+00  ...  -6.4281e-01 -9.1896e-01 -5.8090e-01
 -1.0887e+00 -2.2818e-01  1.1619e+00  ...   7.7911e-02  2.4421e-01 -4.9161e-01
      ⋮  

( 1 , 0 ,.,.) = 
  8.1571e-01  1.2526e+00  6.0463e-01  ...   8.0458e-01  1.1654e+00  1.9355e-01
 -1.5160e+00  1.1829e+00  3.0988e-01  ...   4.2321e-01  7.5024e-01  3.8233e-01
 -1.8064e+00 -3.0960e-01  1.2396e+00  ...  -1.5141e+00 -2.5436e-01  1.5146e+00
                 ...                   ⋱                   ...                
  3.6366e-01 -7.6946e-01  7.5512e-02  ...  -5.0416e-01 -4.5377e-0

In [5]:
b

Variable containing:
( 0 ,.,.) = 
  0.2437 -0.3376 -0.2566  ...   0.2284 -0.5809 -0.4916

( 1 ,.,.) = 
  0.1936  0.3823  1.5146  ...  -1.4903 -1.1073 -0.1940

( 2 ,.,.) = 
 -0.2305 -1.2344  0.7175  ...  -0.3119 -0.8814 -1.4692
... 

(61 ,.,.) = 
  0.2064 -0.1855 -0.6622  ...   0.2029 -1.2677  0.0409

(62 ,.,.) = 
  1.9635 -0.2829  0.5218  ...  -0.4216 -1.0748 -0.3566

(63 ,.,.) = 
 -0.1511 -0.4409  0.7082  ...  -0.3067  0.9117 -0.8072
[torch.FloatTensor of size 64x1x100]

In [6]:
class Net(nn.Module):
    def __init__(self, 
                 vocab_size,
                 embedding_dim,
                 len_sentence,
                 channel_size=4,
                 x2_size=1, # additional data - cap ratio
                 fc_dim=128,
                 padding_idx=1,
                 dropout=0.3,
                 num_labels=7,
                 batch_size=32,
                 is_cuda=False
                ):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(vocab_size+2, embedding_dim=embedding_dim, padding_idx=padding_idx)
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.channel_size = channel_size
        self.len_sentence = len_sentence
        self.batch_size = batch_size
        self.x2_size = x2_size
        
        self.conv2d = nn.Conv2d(1, out_channels=channel_size, kernel_size=(5, embedding_dim + 1), stride=1)
        # output : batch x channel x (len_sentence - 2) x 1
        
        # -> squeeze : batch x channel x (len_sentence - 2)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1d = nn.Dropout(p=dropout)
        self.pool1d = nn.AvgPool1d(kernel_size=2)
        # output : batch x channel x (len_sentence - 2) / 2
        
        self.bottleneck_size = channel_size * (len_sentence - 4) / 2
        assert self.bottleneck_size.is_integer()
        self.bottleneck_size = int(self.bottleneck_size) + self.x2_size
        
        self.fcn1 = nn.Linear(self.bottleneck_size, fc_dim)
        self.relu1 = nn.ReLU(inplace=True)
        self.fcn2 = nn.Linear(fc_dim, num_labels)
        self.sigmoid = nn.Sigmoid()
        self.fc_dim = fc_dim
        self.num_labels = num_labels
    
    def forward(self, sentence, cap_ratio, other_features):
#         print("sentence ", sentence.shape)
        image = self.embedding(sentence)
#         print(bottleneck.shape)
#         image.unsqueeze_(1)
#         print("image ", image.shape)
        # batch x channel x sentence_length x embedding
        cap_ratio.unsqueeze_(2)
#         print("cap_Ratio :", cap_ratio.shape)
        new_image = torch.cat([image, cap_ratio], dim=2)
        new_image.unsqueeze_(1)
        bottleneck = self.conv2d(new_image)
        bottleneck.squeeze_(3)
        bottleneck = self.relu(bottleneck) # batch x channel x features
        bottleneck = self.dropout1d(bottleneck)
        bottleneck = self.pool1d(bottleneck)
#         print("bt shape ", bottleneck.shape)
        
        bottleneck = bottleneck.view(-1, self.bottleneck_size - self.x2_size)
        if self.x2_size > 0:
            bottleneck = torch.cat([bottleneck, other_features], dim=1)
        
        fcn = self.relu1(self.fcn1(bottleneck))
        fcn = self.fcn2(fcn)
        logit = self.sigmoid(fcn)
        
        return logit

In [7]:
class config:
    vocab_size = 20000
    embedding_dim = 100 # TODO: max 300
    len_sentence = 100
    num_labels = 6
    min_freq = 1
    batch_size = 64
    channel_size = 128
    seed = 0
    dropout = 0.5 # TODO: batch norm으로 대체 추천
    x2_size = 0
    valid_num = 10000

In [8]:

# seed 고정
torch.cuda.manual_seed_all(config.seed)
torch.manual_seed(config.seed)
random.seed(config.seed)
np.random.seed(config.seed)
torch.backends.cudnn.deterministic=True


In [9]:
def get_pd_data(path : str):
    df = pd.read_csv(path)
    return df

In [10]:
train = get_pd_data('./data/train.csv')

In [11]:
test = get_pd_data('./data/test.csv')

In [12]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


## Preprocess (1)
----
###  Set captial character ratio
- 문장 내의 대문자 비율을 뉴럴넷의 input으로 줌

In [13]:
def set_capital_ratio(df : pd.DataFrame):
    df['alphas'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isalpha()))
    df['capitals'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['cap_ratio'] = df.apply(lambda row: float(row['capitals']) / (float(row['alphas']) + 1), axis=1)


In [14]:
set_capital_ratio(train), set_capital_ratio(test)

(None, None)

In [15]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,alphas,capitals,cap_ratio
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,203,17,0.083333
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,73,8,0.108108
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,186,4,0.021390
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,486,11,0.022587
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,50,2,0.039216


## Preprocess(2)
-----
### Word tokenize
- gensim의 tokenize function

In [16]:
from gensim.utils import simple_tokenize

C:\Users\sweetcocoa\Anaconda3\envs\torch\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [17]:
def tokenizer(string : str):
    return [s for s in simple_tokenize(string)]

In [18]:
def get_cap_ratio_of_words(tokens : list):
    return [(sum(1 for c in token if c.isupper()))/(sum(1 for c in token if c.isalpha()) + 1) for token in tokens]

In [19]:
tk_train = train['comment_text'].apply(tokenizer)
tk_test = train['comment_text'].apply(tokenizer)
tk_cap_ratio_train = tk_train.apply(get_cap_ratio_of_words)
tk_cap_ratio_test = tk_test.apply(get_cap_ratio_of_words)

In [20]:
tk_train = train['comment_text'].str.lower().apply(tokenizer)
tk_test = train['comment_text'].str.lower().apply(tokenizer)

In [21]:
tk_cap_ratio_train.head()

0    [0.08333333333333333, 0.25, 0.0, 0.0, 0.0, 0.0...
1    [0.5, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, ...
2    [0.25, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3    [0.2, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.25, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0...
Name: comment_text, dtype: object

In [22]:
tk_train[:5]

0    [explanation, why, the, edits, made, under, my...
1    [d, aww, he, matches, this, background, colour...
2    [hey, man, i, m, really, not, trying, to, edit...
3    [more, i, can, t, make, any, real, suggestions...
4    [you, sir, are, my, hero, any, chance, you, re...
Name: comment_text, dtype: object

In [23]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
x_features = ['cap_ratio']

## Preprocess(3)
----
### Add Normal column label
- toxic하지 않은 label로 분류되는 것에, normal=1 의 새로운 라벨 추가 (하지않음)

In [24]:
# train['normal'] = 0
# train.loc[train[labels].sum(axis=1) == 0, 'normal'] = 1

In [25]:
# labels.append('normal')

In [26]:
y_labels = train[labels]
y_labels.head(n=10)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [27]:
# Add manually engineered features ex) capital ratio of sentence
x2 = train[x_features]
x2.head(n=10)

,cap_ratio
0,0.083333
1,0.108108
2,0.021390
3,0.022587
4,0.039216
5,0.021277
6,0.973684
7,0.043478
8,0.019391
9,0.033333


## Validation
----
### 10000 개의 Validation set

In [28]:
# x, y, z : pd.Dataframe of pd.series
def shuffle_lists(*pargs):
    shuffler = np.random.permutation(len(pargs[0]))
    retargs = []
    for x in pargs:
        x = x.iloc[shuffler]
        retargs.append(x)
    return retargs

In [29]:
def padding_cap_ratio(tokens : list):
    L = len(tokens)
    ret = tokens[:min(L, config.len_sentence)]
    ret = [0] * (config.len_sentence - L) + ret
    return np.array(ret)

In [30]:
tk_train, x2, tk_cap_ratio_train, y_labels = shuffle_lists(tk_train, x2, tk_cap_ratio_train, y_labels)

In [31]:
tk_cap_ratio_train = np.array([padding_cap_ratio(row) for row in tk_cap_ratio_train])

In [32]:
tk_valid = tk_train[-config.valid_num:]
tk_cap_ratio_valid = tk_cap_ratio_train[-config.valid_num:]
x2_valid = x2[-config.valid_num:]
y_valid = y_labels[-config.valid_num:]

tk_train = tk_train[:-config.valid_num]
y_train = y_labels[:-config.valid_num]
x2_train = x2[:-config.valid_num]
tk_cap_ratio_train = tk_cap_ratio_train[:-config.valid_num]

In [33]:
len(tk_valid), len(tk_cap_ratio_valid), len(x2_valid), len(y_valid)

(10000, 10000, 10000, 10000)

In [34]:
len(tk_train), len(tk_cap_ratio_train), len(x2_train), len(y_train)

(149571, 149571, 149571, 149571)

In [35]:
from torchtext import data, datasets

## Preprocess(3)
---
### torchtext.data.Field
- word dictionary, word to index 구현

In [36]:
TEXT = data.Field(sequential=True,  
                  # 들어갈 데이터가 sequential 인가요? 우리는 tokenize한 word의 sequence를 다룰거니까 True입니다. Defualt로도 True임.
                  tokenize=tokenizer, 
                  # 그 데이터를 tokenize할 함수를 지정할 수 있습니다. 우리는 gensim library의 tokenize 함수를 쓸건데요
                  # 뭐 굳이 그거 말고도 직접 정의해도 되고 str.split 같은걸 써넣어도 됩니다.
                  # :: 그런 줄 알았는데 아무 tokenize 함수나 쓰면 안되고, generator가 아닌 tokenized list 를 반환하는 함수여야합니다..
                  # :::: 이게 아닐거같기도 함.
                  fix_length=config.len_sentence,
                 # 아마 tokenize된 길이 제한 같은데 한번 확인해볼게요. 특이사항으로는 length 넘으면 자르고, 안넘으면 padding을 채웁니다
                  # :: 그게 아니고 vector화 했을 때의 길이 제한일 것 같아요. 확인해보겠습니다.
                  pad_first=True,
                  # padding이 앞에서부터 붙냐, 뒤에서부터 붙냐는 겁니다.
                  tensor_type=torch.cuda.LongTensor
                  # cuda를 써도 됩니다
                 )

In [37]:
TEXT.build_vocab(tk_train, tk_valid, max_size=config.vocab_size, min_freq=config.min_freq)

In [38]:
def batchify(*pargs, batch_size=32):
    for i in range(0, len(pargs[0]), batch_size):
        end = min(i+batch_size, len(tk_train))
        yield [batch[i:end] for batch in pargs]

In [39]:
net = Net(vocab_size=config.vocab_size, embedding_dim=config.embedding_dim, len_sentence=config.len_sentence,
         x2_size=config.x2_size, channel_size=config.channel_size, dropout=config.dropout, num_labels=config.num_labels, batch_size=config.batch_size).cuda()

In [40]:
net

Net(
  (embedding): Embedding(20002, 100, padding_idx=1)
  (conv2d): Conv2d (1, 128, kernel_size=(5, 101), stride=(1, 1))
  (relu): ReLU(inplace)
  (dropout1d): Dropout(p=0.5)
  (pool1d): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,), ceil_mode=False, count_include_pad=True)
  (fcn1): Linear(in_features=6144, out_features=128)
  (relu1): ReLU(inplace)
  (fcn2): Linear(in_features=128, out_features=6)
  (sigmoid): Sigmoid()
)

In [41]:
optimizer = optim.Adam(net.parameters())
criterion = nn.BCELoss()
# criterions = [nn.CrossEntropyLoss() for i in range(config.num_labels)]
# -> Binary

In [42]:
from tqdm import tqdm

In [43]:
def validation(net, tk_valid, tk_cap_ratio_valid, x2_valid, y_valid : pd.DataFrame, TEXT : data.Field, criterion):
    net.train(False)
    val_score = None
    valid_loss = 0 
    for val_step, (batch_val, cr_val, x2_val, y_val) in enumerate(batchify(tk_valid, tk_cap_ratio_valid, x2_valid.values, y_valid.values, batch_size=config.batch_size)):
        var_x = TEXT.process(batch_val, device=0, train=False).transpose(dim0=0, dim1=1)
        var_cr = Variable(torch.cuda.FloatTensor(cr_val))
        var_y = Variable(torch.cuda.FloatTensor(y_val))
        var_x2 = Variable(torch.cuda.FloatTensor(x2_val))
        pred_score = net(var_x, var_cr, var_x2)
        if val_score is None:
            val_score = pred_score
        else:
            val_score = torch.cat([val_score, pred_score])
        y_loss = criterion(pred_score, var_y)
        valid_loss += y_loss.data[0]
    net.train(True)
    valid_loss /= len(tk_valid) / config.batch_size
            
    return valid_loss, val_score

In [59]:
train_corrects = [0 for i in range(config.num_labels)]
train_loss = 0
net.train(True)
for step, (x_train_, cr_train_, x2_train_, y_train_) in tqdm(enumerate(batchify(tk_train, tk_cap_ratio_train, x2_train.values, y_train.values, batch_size=config.batch_size))):
    var_x = TEXT.process(x_train_, device=0, train=True).transpose(dim0=0, dim1=1)
    var_cr = Variable(torch.cuda.FloatTensor(cr_train_))
    var_y = Variable(torch.cuda.FloatTensor(y_train_))
    var_x2 = Variable(torch.cuda.FloatTensor(x2_train_))
    pred_score = net(var_x, var_cr, var_x2)
    
    net.zero_grad()
    
    
    y_loss = criterion(pred_score, var_y)
#     if step % 100 == 99:
#         print(y_loss.data[0])
    y_loss.backward()
    if step % 1000 == 999:
        valid_loss, val_score = validation(net, tk_valid, tk_cap_ratio_valid, x2_valid, y_valid, TEXT, criterion)
        print("valid loss", valid_loss)
#         print("valid acc", [i.data[0] for i in valid_acc])

    optimizer.step()
    

996it [00:09, 102.86it/s]

valid loss 0.05752939092963934


1997it [00:20, 99.21it/s]

valid loss 0.05879605170637369


2338it [00:24, 96.15it/s]


In [60]:
valid_loss, val_score = validation(net, tk_valid, tk_cap_ratio_valid, x2_valid, y_valid, TEXT, criterion)

In [61]:
from sklearn.metrics import roc_auc_score


## TODO
---
### roc_auc_score w.r.t. validation set's score
- Kaggle form에 맞추어 column-wise roc auc score 계산

In [62]:
roc_auc_scores = 0
for i in range(config.num_labels): # minus 1 for normal 
    score = roc_auc_score( y_valid[labels[i]].values, val_score.data.cpu().numpy()[:, i])
    print(score)
    roc_auc_scores += score

0.965691001184
0.985493937663
0.981482735949
0.973019586525
0.975710950977
0.957344804756


In [63]:
print(roc_auc_scores / (config.num_labels))

0.973123836176


- 0.975129977385 - CHANNEL 128 + PURE RELU between FCN + PURE RELU between CNN + dropout 0.5 + AvgPool1d between CNN + sentence length 100 + embedding 100 + min_freq 1 + kernel size 5 + epoch 3
- 0.971308288115 - same above + word capital embedding + epoch 2
- 0.973348269027 - same above + epoch 3
- 0.973123836176 - same above + epoch 4